In [1]:
import os
import ast
import torch
import torch.nn as nn

from model import bengalimodel
from dataset import BengaliDatasetTrain
from train_fn import train,evaluate

In [2]:
device = 'cuda'
IMG_HEIGHT=137
IMG_WIDTH=236
EPOCHS=25
TRAIN_BATCH_SIZE=64
TEST_BATCH_SIZE=16
MODEL_MEAN=(0.485, 0.456, 0.406)
MODEL_STD=(0.229, 0.224, 0.225)

In [3]:
TRAINING_FOLDS = [1,2,3,4]
VALIDATION_FOLDS = [0]

model = bengalimodel(backbone = 'resnet18')
model = model.to(device)

train_dataset = BengaliDatasetTrain(
    folds=TRAINING_FOLDS,
    img_height = IMG_HEIGHT,
    img_width = IMG_WIDTH,
    mean = MODEL_MEAN,
    std = MODEL_STD
)

train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size= TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=4
)

valid_dataset = BengaliDatasetTrain(
    folds=VALIDATION_FOLDS,
    img_height = IMG_HEIGHT,
    img_width = IMG_WIDTH,
    mean = MODEL_MEAN,
    std = MODEL_STD
)

valid_loader = torch.utils.data.DataLoader(
    dataset=valid_dataset,
    batch_size= TEST_BATCH_SIZE,
    shuffle=True,
    num_workers=4
)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
optimizer,
mode = 'min',
patience = 5,
factor = 0.3,
verbose = True
)



Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [4]:
best_score = -1
es = 0
for epoch in range(EPOCHS):
    train_loss, train_score = train(train_dataset,train_loader, model, optimizer)
    val_loss, val_score = evaluate(valid_dataset, valid_loader, model)
    
    scheduler.step(val_loss)

    
    if val_score > best_score:
            best_score = val_score
            es = 0
            torch.save(model.state_dict(), f"resnet18_fold{VALIDATION_FOLDS[0]}.pth")
    else:
        es += 1

    epoch_len = len(str(EPOCHS))
    print_msg = (f'[{epoch:>{epoch_len}}/{EPOCHS:>{epoch_len}}] ' +
                 f'train_loss: {train_loss:.5f} ' +
                 f'train_score: {train_score:.5f} ' +
                 f'valid_loss: {val_loss:.5f} ' +
                 f'valid_score: {val_score:.5f}'
                )

    print(print_msg)
    if es > 4:
        print("Early stopping")
        break

2511it [04:55,  8.50it/s]                          


=================Train=================
recall: grapheme 0.8044096895662305, vowel 0.9098206178912704, consonant 0.9082203933649018, total 0.8567150975971582


2511it [01:41, 24.80it/s]                          


=================VALID============
recall: grapheme 0.8980004275863047, vowel 0.9608997183843879, consonant 0.9516139165815236, total 0.9271286225346302
[ 0/25] train_loss: 0.19238 train_score: 0.85672 valid_loss: 0.09246 valid_score: 0.92713


2511it [04:55,  8.50it/s]                          


...

=================VALID============
recall: grapheme 0.9490589998771436, vowel 0.9822289124699154, consonant 0.9672586875057357, total 0.9619013999324846
[14/25] train_loss: 0.00073 train_score: 0.99936 valid_loss: 0.07072 valid_score: 0.96190



2511it [04:55,  8.49it/s]                          


=================Train=================
recall: grapheme 0.9995318293260321, vowel 0.9993797740157163, consonant 0.9991470879353974, total 0.9993976301507945


2511it [01:41, 24.73it/s]                          

=================VALID============
recall: grapheme 0.9511367052931412, vowel 0.9804174625160738, consonant 0.9765127422296646, total 0.9648009038330051
Epoch    16: reducing learning rate of group 0 to 9.0000e-06.
[15/25] train_loss: 0.00072 train_score: 0.99940 valid_loss: 0.07302 valid_score: 0.96480
Early stopping
